> "Context managers allow you to allocate and release resources precisely when you want to. The most widely used example of context managers is the with statement. Suppose you have two related operations which you’d like to execute as a pair, with a block of code in between. Context managers allow you to do specifically that." --<cite> book.pythontips.com </cite>

For example the common patterns: 

``` 
f = open(...)    ## File managment 
...
f.close()

lock.acquire()   ## Multhreaded application 
...
lock.release()

start = time.time() ## Timing code execution 
...
end = time.time()
```

You can read and close a file with a ``try/except`` block:  

In [ ]:
try:
    f = open('uchicago-emails.txt', 'r')
    for email in f: 
        print(email)
finally:
    f.close()

But we don't. Instead, you'll more often see this, which closes the file for you when your operation works...

In [ ]:
## More compact and readable 
with open('uchicago-emails.txt', 'r') as f: 
    for email in f: 
        print(email)

...and when it doesn't.

In [1]:
with open('not_a_real_file.txt', 'r') as here:
    for nothing in here:
        print(nothing)

FileNotFoundError: [Errno 2] No such file or directory: 'not_a_real_file.txt'

It's called a **context manager** because it wraps your code in a box (a context). The box is opened, the code is run, and then the box is shut. Regardless of what happens when the code runs, the box is always shut. That way, if the code in the box explodes, you can shut the box to clean up the mess!

The ``File`` class is a context manager because it implements the **context manager protocol**.

Any class can be a context manger. A class must define two methods: 

- ``__enter__(self)`` is executed at the start of a with block. It needs to return the context manager. 
- ``__exit__(self, type, value, traceback)`` is executed at the end of a with block and performs any "cleanup" actions. If there is an exception that occurs between the ``__enter__`` and the calling of ``__exit__`` then python calls ``__exit__`` with the information about the exception via these three parameters. This step allows the ``__exit__`` method to decide how to hanlde the exception and if any further steps are required to cleanup. 

Lets make our own context manager class that handles testing the amount of time a piece of code takes to complete.


In [7]:
from timeit import default_timer as timer

class Stopwatch:
    
    def __init__(self):
        self.start_time = 0 
        self.end_time = 0 
        
    def __enter__(self): 
        self.start_time = timer() 
        return self 
    
    def __exit__(self, type, value, traceback):
        self.end_time = timer() 
        
    @property 
    def elasped_time(self):
        return self.end_time-self.start_time 

In [8]:
def fibonacci(n): 
    if n==1: 
        return 0
    elif n==2: 
        return 1
    else: 
        return fib(n-1)+fib(n-2) 

In [9]:
with Stopwatch() as s: 
    fibonacci(35)
print(s.elasped_time)

2.826494860999901


More information about context managers can be found here: https://docs.python.org/3/library/contextlib.html